In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

# Load dataset
df = pd.read_csv('appointments.csv')

# Dates
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'])
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'])

# Feature Engineering
df['DayGap'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days
df['AppointMonth'] = df['AppointmentDay'].dt.month
df['AppointWeekDay'] = df['AppointmentDay'].dt.weekday
df['IsWeekend'] = df['AppointWeekDay'].apply(lambda x: 1 if x >= 5 else 0)
df['ScheduleHour'] = df['ScheduledDay'].dt.hour
df['WaitingTime'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.total_seconds() / (60*60*24)

# Patient past behavior features
df['No-show'] = df['No-show'].map({'Yes': 1, 'No': 0})
df['PatientNoShowRate'] = df.groupby('PatientId')['No-show'].transform(lambda x: x.rolling(5, min_periods=1).mean())
df['PatientTotalAppointments'] = df.groupby('PatientId')['No-show'].transform('count')
df['PatientMissedBefore'] = df.groupby('PatientId')['No-show'].transform('sum')

# Drop leakage or irrelevant
df = df[df['DayGap'] >= 0]
df.drop(['ScheduledDay', 'AppointmentDay', 'AppointmentID'], axis=1, inplace=True)

# Categorical encoding
df = pd.get_dummies(df, columns=['Gender', 'Neighbourhood'], drop_first=True)

# Drop any remaining NaNs
df.fillna(0, inplace=True)

# Features & Target
X = df.drop(['No-show', 'PatientId'], axis=1)
y = df['No-show']

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Balance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Train-Test
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42
)

# Base Models
xgb = XGBClassifier(n_estimators=300, max_depth=7, learning_rate=0.03, subsample=0.9,
                    colsample_bytree=0.8, use_label_encoder=False, eval_metric='logloss', random_state=42)

rf = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42)

gb = GradientBoostingClassifier(n_estimators=250, learning_rate=0.05, max_depth=5, random_state=42)

# Meta Model
lr = LogisticRegression()

# Stacking Ensemble
stacked_model = StackingClassifier(
    estimators=[('xgb', xgb), ('rf', rf), ('gb', gb)],
    final_estimator=lr,
    cv=5
)

# Train Ensemble
stacked_model.fit(X_train, y_train)

# Predict
y_pred = stacked_model.predict(X_test)

# Evaluate
print("\nENSEMBLE Accuracy:", round(accuracy_score(y_test, y_pred) * 100, 2), "%")
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Score All
df['RiskScore'] = stacked_model.predict_proba(scaler.transform(X))[:, 1]
df['HighRisk'] = df['RiskScore'] > 0.6
df['Recommendation'] = np.where(df['HighRisk'], 'Send Reminder / Offer Transport', 'Standard Follow-up')

print("\n🎯 Top Rows:\n", df[['RiskScore', 'HighRisk', 'Recommendation']].head())



ENSEMBLE Accuracy: 95.33 %

Confusion Matrix:
 [[9747  541]
 [ 420 9867]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.95      0.95     10288
           1       0.95      0.96      0.95     10287

    accuracy                           0.95     20575
   macro avg       0.95      0.95      0.95     20575
weighted avg       0.95      0.95      0.95     20575


🎯 Top Rows:
     RiskScore  HighRisk                   Recommendation
5    0.004478     False               Standard Follow-up
6    0.990343      True  Send Reminder / Offer Transport
7    0.990433      True  Send Reminder / Offer Transport
9    0.004471     False               Standard Follow-up
10   0.004471     False               Standard Follow-up
